In [103]:
!pip install torch_geometric
!pip install spektral

In [104]:
import numpy as np
import tensorflow as tf
import torch as th
from spektral.layers.convolutional import ARMAConv as ARMAConvSpektral
from torch_geometric.nn import ARMAConv as ARMAConvTorch
from torch_geometric.data import Data

from torch_geometric.loader import DataLoader
from torch.utils.data import TensorDataset

from spektral.layers import ARMAConv
from tensorflow.keras import Model
from tensorflow.keras.layers import Input
from spektral.data import Graph,BatchLoader,Dataset


In [105]:

# Create a sample adjacency matrix (replace this with your actual data)
adjacency_matrix = np.array([[[0, 1, 0],
                             [1, 0, 1],
                             [0, 1, 0]],[[0, 1, 0],
                             [1, 0, 1],
                             [0, 1, 0]]])

# Save the adjacency matrix as adjacency_df.npy
np.save("adjacency_df.npy", adjacency_matrix)
a_input = np.load('adjacency_df.npy')

In [106]:
a_input

array([[[0, 1, 0],
        [1, 0, 1],
        [0, 1, 0]],

       [[0, 1, 0],
        [1, 0, 1],
        [0, 1, 0]]])

In [107]:
def array_matrices_to_array_edge_index(array_matrices: np.ndarray):
    print(array_matrices.shape)
    array_edge_indices = []
    array_edge_indices_weighted = []

    # Iterate over the adjacency matrices
    for matrix in array_matrices:
        edge_indices = []
        edge_weights = []
        for i in range(matrix.shape[0]):
            for j in range(matrix.shape[1]):
                if matrix[i, j] != 0:
                    # Add the edge indices and the weight
                    edge_indices.append([i, j])
                    edge_weights.append(matrix[i, j])

        #the edge indices and weights should always contain 3 columns, fill with [] if not

        edge_indices = np.array(edge_indices)
        edge_weights = np.array(edge_weights)
        array_edge_indices.append(edge_indices)
        array_edge_indices_weighted.append(edge_weights)

    return np.array(array_edge_indices), np.array(array_edge_indices_weighted)

In [108]:
array_matrices_to_array_edge_index(a_input[1:2])

(1, 3, 3)


(array([[[0, 1],
         [1, 0],
         [1, 2],
         [2, 1]]]),
 array([[1, 1, 1, 1]]))

In [109]:
node_features = th.tensor([
    [1.0, 2.0, 3.0],
    [1.0, 2.0, 3.0],
    [1.0, 2.0, 3.0],
])

edge_indices = th.tensor(
    [
        [0, 1, 1, 1],  # Source nodes
        [0, 2, 0, 1],  # Target nodes
    ]

)



# Adjacency matrix
# [1., 3., 0.],
# [3., 4., 2.],
# [0., 2., 0.],


edge_weights = th.tensor([
    1., 2., 3., 4.
])

print(node_features.shape)
print(edge_indices.shape)
print(edge_weights.shape)

arma = ARMAConvTorch(3, 3, shared_weights=True)
arma(node_features, edge_indices, edge_weight=edge_weights)

torch.Size([3, 3])
torch.Size([2, 4])
torch.Size([4])


tensor([[0.7427, 0.0000, 0.0000],
        [0.7427, 0.0000, 0.0000],
        [0.4216, 0.0000, 0.0000]], grad_fn=<MeanBackward1>)

In [110]:
# node_features = tf.convert_to_tensor([
#     [
#         [1., 0., 0.],
#         [0., 0., 0.],
#         [0., 0., 0.],
#     ],
#     [
#         [0., 0., 0.],
#         [0., 2., 0.],
#         [0., 0., 0.],
#     ]
# ]
#
# )
#
adjacency_graph = tf.convert_to_tensor(
    np.array([
        [
            [1., 0., 0.,2.],
            [0., 1., 0.,2.],
            [0., 0., 1.,2.]
        ],
        [
            [0., 2., 3.,5.],
            [2., 0., 4.,0.],
            [3., 4., 0.,0.],

        ],
    ]
    )
)

node_features = tf.convert_to_tensor(
    [
        [1.0, 2.0, 3.0],
        [1.0, 2.0, 3.0],
        [1.0, 2.0, 3.0],
    ]
)

adjacency_graph = tf.convert_to_tensor(

    [
        [1., 3., 0.],
        [3., 4., 2.],
        [0., 2., 0.],
    ],

)

print(node_features.shape)
print(adjacency_graph.shape)

arma = ARMAConvSpektral(node_features.shape[-1], shared_weights=True)
arma([node_features, adjacency_graph])

(3, 3)
(3, 3)


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[14.34095  ,  4.0955806,  0.       ],
       [28.681902 ,  8.191161 ,  0.       ],
       [ 8.60457  ,  2.4573483,  0.       ]], dtype=float32)>

In [111]:
import torch
from torch_geometric.nn import ARMAConv
from torch_geometric.data import Data

# Sample data preparation
num_nodes = 10
num_node_features = 5
num_classes = 3
edges = torch.tensor([[0, 1, 1, 2,2],
                      [1, 2, 0, 3,1]], dtype=torch.long)
node_features = torch.tensor([
    [1.0, 2.0, 3.0, 4.0, 10.0],
    [1.0, 2.0, 3.0, 4.0, 10.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
])

# Graph data object
data = Data(x=node_features, edge_index=edges)


# Model
class PyGARMA(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.arma = ARMAConv(num_node_features, num_classes, num_stacks=1, num_layers=1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.arma(x, edge_index)
        return x


model_pyg = PyGARMA()
model_pyg.train()
out = model_pyg(data)
print(out)

tensor([[0.1452, 0.0000, 1.5484],
        [3.2113, 0.0000, 0.0000],
        [2.8205, 0.0000, 0.0000],
        [3.7183, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.5442],
        [0.0000, 0.0000, 1.5442],
        [0.0000, 0.0000, 1.5442],
        [0.0000, 0.0000, 1.5442],
        [0.0000, 0.0000, 1.5442],
        [0.0000, 0.0000, 1.5442]], grad_fn=<MeanBackward1>)


In [112]:
import numpy as np
from spektral.layers import ARMAConv
from tensorflow.keras import Model
from tensorflow.keras.layers import Input
from spektral.data import Graph

# Converting PyTorch data to Spektral graph
num_nodes = 10
node_features = torch.tensor([
    [1.0, 2.0, 3.0, 4.0, 10.0],
    [1.0, 2.0, 3.0, 4.0, 10.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
])
x = np.array(node_features)
a = np.zeros((num_nodes, num_nodes))
a[edges[0], edges[1]] = 1

graph = Graph(x=x, a=a)


# Spektral model
class SpektralARMA(Model):

    def __init__(self):
        super().__init__()
        self.arma = ARMAConv(channels=num_classes, n_stacks=1, n_layers=1)

    def call(self, inputs):
        x, a = inputs
        return self.arma([x, a])


x_in = Input(shape=(num_nodes,num_node_features))
a_in = Input((num_nodes,num_nodes))
model_spektral = SpektralARMA()

print(x_in.shape)
print(a_in.shape)
model_spektral([x_in, a_in])
model_spektral.compile()
out = model_spektral([graph.x, graph.a])
print(out)


(None, 10, 5)
(None, 10, 10)
tf.Tensor(
[[2.6525695 1.6655154 7.0952306]
 [1.3024714 4.112406  9.617671 ]
 [0.        5.7265387 8.592497 ]
 [0.        2.4468904 2.5224407]
 [0.        2.4468904 2.5224407]
 [0.        2.4468904 2.5224407]
 [0.        2.4468904 2.5224407]
 [0.        2.4468904 2.5224407]
 [0.        2.4468904 2.5224407]
 [0.        2.4468904 2.5224407]], shape=(10, 3), dtype=float32)


In [113]:
import torch

# Example adjacency matrices as dense tensors
adj_matrices = torch.tensor([
    [[1., 0., 0.],
     [0., 1., 1.],
     [0., 1., 1.]],

    [[0., 1., 0.],
     [1., 0., 1.],
     [0., 1., 0.]]
])

# Convert adjacency matrices to edge_index format and edge_type
num_nodes = adj_matrices.size(1)
edge_index = []
edge_type = []

for i, adj in enumerate(adj_matrices):
    row, col = torch.where(adj > 0)
    edge_index.append(torch.stack([row, col], dim=0))
    edge_type.append(torch.full((row.size(0),), i, dtype=torch.long))

# Concatenate all edges
edge_index = torch.cat(edge_index, dim=1)
edge_type = torch.cat(edge_type)

print(edge_index)
print(edge_type)


tensor([[0, 1, 1, 2, 2, 0, 1, 1, 2],
        [0, 1, 2, 1, 2, 1, 0, 2, 1]])
tensor([0, 0, 0, 0, 0, 1, 1, 1, 1])


#Teste DataLoader Tensorflow

In [114]:
x = torch.randn(5,5, 3)
y = torch.randn(5,5, 3)
input_train_tensor = [torch.tensor(data, dtype=torch.float32) for data in x]
input_test_tensor = [torch.tensor(data, dtype=torch.float32) for data in y]
data_list = []
for i in range(len(input_train_tensor)):
    num_nodes = input_train_tensor[i].shape[0]
    edge_index = torch.randint(0, num_nodes, (2, num_nodes), dtype=torch.long)
    data = Data(x=input_train_tensor[i], edge_index=edge_index, y=input_test_tensor[i])

    data_list.append(data)




<ipython-input-114-0e892f496142>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_train_tensor = [torch.tensor(data, dtype=torch.float32) for data in x]
<ipython-input-114-0e892f496142>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_test_tensor = [torch.tensor(data, dtype=torch.float32) for data in y]


In [115]:

batch_size = 2
train_loader = DataLoader(data_list, batch_size=batch_size, shuffle=True)
a = 1
for batch in train_loader:
    print(f"Batch: {a}")
    print("x:", batch.x)
    print("y:", batch.y)
    print("edge_index:", batch.edge_index)
    print("Number Nodes:", batch.num_nodes)
    print("Number of graphs in the batch:", batch.num_graphs)
    print("\n\n")

    a = a + 1

Batch: 1
x: tensor([[-0.0391,  0.2706,  1.2297],
        [-0.5794, -0.0197,  0.6737],
        [-0.4616, -0.5692, -0.9983],
        [ 2.7324,  0.8595,  0.5742],
        [-0.2779,  0.4929,  1.3706],
        [-1.9385, -0.6192, -0.0110],
        [ 0.5321, -0.6067, -0.9317],
        [ 1.7746, -1.2165, -0.1197],
        [ 1.3536,  0.1787,  0.0876],
        [-0.2142,  0.6710, -1.1213]])
y: tensor([[-1.8536, -0.2581,  0.7118],
        [-0.5610, -0.7685, -0.2310],
        [ 0.6411,  0.0025, -0.8139],
        [-0.6762, -1.5751,  0.1968],
        [ 0.3594,  0.9562,  0.2851],
        [-1.1048, -0.0588, -0.3000],
        [-0.3751, -0.3823, -1.5682],
        [-0.8514, -1.6073,  1.6078],
        [-0.6608,  1.8865, -0.6165],
        [ 1.1471,  0.7812,  0.8736]])
edge_index: tensor([[2, 3, 2, 1, 1, 8, 5, 7, 8, 6],
        [0, 1, 1, 0, 2, 7, 7, 7, 9, 7]])
Number Nodes: 10
Number of graphs in the batch: 2



Batch: 2
x: tensor([[-1.4461,  0.2262,  1.1151],
        [-0.9588, -0.7632,  0.9383],
        [-0

In [116]:
batch_size = len(input_train_tensor)
train_loader = DataLoader(data_list, batch_size=batch_size, shuffle=True)
a = 1
for batch in train_loader:
    print(f"Batch: {a}")
    print("x:", batch.x)
    print("y:", batch.y)
    print("edge_index:", batch.edge_index)
    print("Number Nodes:", batch.num_nodes)
    print("Number of graphs in the batch:", batch.num_graphs)
    print("\n\n")

    a = a + 1

Batch: 1
x: tensor([[-1.4461,  0.2262,  1.1151],
        [-0.9588, -0.7632,  0.9383],
        [-0.4256,  0.3079, -0.3824],
        [-0.9715,  0.6375,  0.6618],
        [ 1.1209,  0.8293,  1.2973],
        [-0.0391,  0.2706,  1.2297],
        [-0.5794, -0.0197,  0.6737],
        [-0.4616, -0.5692, -0.9983],
        [ 2.7324,  0.8595,  0.5742],
        [-0.2779,  0.4929,  1.3706],
        [-1.9385, -0.6192, -0.0110],
        [ 0.5321, -0.6067, -0.9317],
        [ 1.7746, -1.2165, -0.1197],
        [ 1.3536,  0.1787,  0.0876],
        [-0.2142,  0.6710, -1.1213],
        [ 1.1104,  0.5899, -0.6387],
        [-0.1596, -0.7393, -1.9666],
        [ 0.6092,  0.2994, -1.6205],
        [-0.7851,  1.4123,  0.3493],
        [ 0.3006,  0.5886, -0.4960],
        [ 1.9902, -1.1630,  0.4580],
        [-1.3442,  0.3708, -1.3309],
        [ 0.7602,  3.1458,  1.3889],
        [-1.0820, -0.6834,  1.0548],
        [-0.5559,  0.0915,  0.5616]])
y: tensor([[ 2.9920e-01,  6.1642e-02, -1.0864e+00],
        [ 

In [117]:
import torch.nn.functional as F
import torch.nn as nn

for batch in train_loader:
  arma = ARMAConvTorch(-1, 5, shared_weights=True, act=nn.GELU())
  x=arma(batch.x, batch.edge_index, edge_weight=batch.edge_attr)
  x = F.elu(x)
  print(x)
  print("\n\n")
  x =  F.softmax(x,dim=1)

  print(x)

tensor([[-0.1203, -0.1539, -0.1121, -0.1512,  1.2015],
        [ 0.0078,  0.2753,  0.1583,  1.2015, -0.0834],
        [ 1.8274,  1.8097,  0.2025,  1.3226, -0.1264],
        [ 0.3425,  0.5219,  0.0662,  0.3362, -0.1557],
        [-0.1071, -0.1327,  0.4532,  0.2748, -0.1502],
        [-0.0202,  0.1080,  0.5138,  0.6812, -0.1161],
        [-0.1519, -0.1147,  0.4300,  1.8222, -0.1056],
        [-0.1360, -0.1506,  0.6209,  1.2996, -0.1226],
        [-0.1343, -0.0974,  0.0960, -0.1039, -0.1497],
        [-0.0745,  0.0048,  0.2900,  0.3788, -0.1561],
        [ 1.7491,  2.2444, -0.1339,  0.9757,  0.2532],
        [-0.0630, -0.0758,  0.3754,  0.1144, -0.1122],
        [ 0.1182, -0.1504,  1.7493, -0.0033, -0.0082],
        [ 0.6466,  0.2444, -0.1558, -0.1478,  1.3963],
        [ 0.0534, -0.0686, -0.0954, -0.1563,  0.2944],
        [-0.1336, -0.1547, -0.1559, -0.0613,  1.2257],
        [ 0.6762,  0.3170, -0.1548, -0.1561,  1.3420],
        [-0.1555, -0.1551,  0.1183, -0.0334, -0.0892],
        [-

#Teste BatchLoader Tensorflow

In [118]:


num_nodes = 10
node_features = torch.tensor([
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
])
edges = torch.tensor([[0, 1, 1, 2,2],
                      [1, 2, 0, 3,1]], dtype=torch.long)
x = np.array(node_features)
a = np.zeros((num_nodes, num_nodes))
a[edges[0], edges[1]] = 1

graph = Graph(x=x, a=a)

class MyDataset(Dataset):
    def __init__(self, graphs):
        self.graphs = graphs

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        return self.graphs[idx]

dataset = MyDataset([graph for x in range(20)])

loader = BatchLoader(dataset, batch_size=10, epochs=1)


for x, batch in enumerate(loader):
    print(f"Batch {x+1}:\n\n")
    inputs, target = batch
    print(f"Input: {inputs}\n\nTarget:{target}")
    print("\n\n")

Batch 1:


Input: [[[1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]]

 [[1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]]

 [[1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]]

 [[1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]]

 [[1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]]

 [[1. 2. 3. 4. 5.]
  [

In [119]:
loader = BatchLoader(dataset, batch_size=20, epochs=1)


for x, batch in enumerate(loader):
    print(f"Batch {x+1}:\n\n")
    inputs, target = batch
    print(f"Input: {inputs}\n\nTarget:{target}")
    print("\n\n")

Batch 1:


Input: [[[1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]]

 [[1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]]

 [[1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]]

 [[1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]]

 [[1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]
  [1. 2. 3. 4. 5.]]

 [[1. 2. 3. 4. 5.]
  [